In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import Loss

In [16]:
# Retrieve DAX data from Yahoo Finance
data = yf.download("^GDAXI", start="2020-01-01", end="2023-01-01")
data = data[['Close']].values


[*********************100%%**********************]  1 of 1 completed


In [17]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [24]:
# Assuming we're using the last 60 days to predict the next 7 days
X = []
y = []
n_future = 7  # Number of days we want to predict into the future
n_past = 60  # Number of past days we want to use to predict the future
n_quantiles = 5  # Number of quantiles we want to predict

for i in range(n_past, len(data_scaled) - n_future +1):
    X.append(data_scaled[i - n_past:i, 0:data_scaled.shape[1]])
    future_quantiles = []
    for j in range(n_future):
        # Here you should have a logic to get the 'n_quantiles' quantiles for day 'i+j'
        # For example, this could be a placeholder for the actual quantile values you expect to predict
        # This is where you would implement your quantile prediction logic
        quantiles_for_day = np.quantile(n_quantiles)  # Placeholder for actual quantile predictions
        future_quantiles.extend(quantiles_for_day)
    y.append(future_quantiles)


X, y = np.array(X), np.array(y)


In [25]:
n_horizons = 5  # Number of horizons you are predicting
n_quantiles = 7  # Number of quantiles you are predicting for each horizon

In [26]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(n_past, data_scaled.shape[1])),
    LSTM(units=50),
    Dense(n_horizons * n_quantiles)  # Output layer with units for each quantile of each horizon
])


2023-11-05 11:51:32.046909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:51:32.048310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:51:32.049090: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [27]:

# Custom loss function for quantiles
class QuantileLoss(Loss):
    def __init__(self, quantiles):
        super().__init__()
        self.quantiles = quantiles

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        loss = 0
        for k in self.quantiles:
            loss += tf.reduce_mean(tf.maximum(k * error, (k - 1) * error), axis=-1)
        return loss

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

In [28]:
# Compile the model
model.compile(optimizer='adam', loss=QuantileLoss(quantiles))

In [29]:
# Fit the model
model.fit(X, y, epochs=5, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/5


2023-11-05 11:51:35.590480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:51:35.591521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:51:35.592357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ValueError: in user code:

    File "/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/y3/h1kjcg6j0wj3crmk4m1ff0380000gn/T/ipykernel_52558/766786216.py", line 8, in call  *
        error = y_true - y_pred

    ValueError: Dimensions must be equal, but are 7 and 35 for '{{node QuantileLoss/sub}} = Sub[T=DT_FLOAT](IteratorGetNext:1, sequential_4/dense_4/BiasAdd)' with input shapes: [?,7], [?,35].


In [ ]:
# Predict the quantiles
predicted_quantiles = model.predict(X[-1].reshape(1, n_past, data_scaled.shape[1]))

In [ ]:
# Reshape the predictions to have the quantiles for each day
predicted_quantiles = predicted_quantiles.reshape(n_future, len(quantiles))

# Inverse transform the predictions to get actual values
predicted_quantiles = scaler.inverse_transform(predicted_quantiles)

In [3]:
# Create the submission DataFrame
submission = pd.DataFrame(data=predicted_quantiles, columns=[f'q{q}' for q in quantiles])
submission['target'] = 'DAX'
submission['horizon'] = [f'{i+1} day' for i in range(n_future)]
submission = submission[['target', 'horizon'] + [f'q{q}' for q in quantiles]]

# Save the DataFrame to a CSV file
submission.to_csv("DAX_quantile_predictions.csv", index=False)


[*********************100%%**********************]  1 of 1 completed


2023-11-05 11:38:48.267307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:38:48.268484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:38:48.269150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/50


2023-11-05 11:38:48.608777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:38:48.609926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:38:48.610785: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ValueError: in user code:

    File "/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/y3/h1kjcg6j0wj3crmk4m1ff0380000gn/T/ipykernel_52558/578817440.py", line 43, in call  *
        error = y_true - y_pred

    ValueError: Dimensions must be equal, but are 7 and 35 for '{{node QuantileLoss/sub}} = Sub[T=DT_FLOAT](IteratorGetNext:1, sequential/dense/BiasAdd)' with input shapes: [?,7], [?,35].
